In [1]:
import torch
import torchvision
import torchvision.datasets as datasets
import torchvision.transforms as transforms

In [6]:
transform = transforms.Compose([transforms.ToTensor(),
                             transforms.Normalize((0.5,),(0.5,))])

In [7]:
train_data = datasets.MNIST(root='data', train=True, download=True, transform=transform)
test_data = datasets.MNIST(root='data', train=False, download=True, transform=transform)

100.1%

Extracting data/MNIST/raw/train-images-idx3-ubyte.gz


113.5%

Extracting data/MNIST/raw/train-labels-idx1-ubyte.gz


100.4%

Extracting data/MNIST/raw/t10k-images-idx3-ubyte.gz


180.4%

Extracting data/MNIST/raw/t10k-labels-idx1-ubyte.gz
Processing...
Done!


In [9]:
from torch.utils.data import Subset


In [10]:
num_teachers = 100 # Define the num of teachers
batch_size = 32 # Teacher batch size

def get_data_loaders(train_data, num_teachers):
    """ Function to create data loaders for the Teacher classifier """
    teacher_loaders = []
    data_size = len(train_data) // num_teachers
    
    for i in range(data_size):
        indices = list(range(i*data_size, (i+1)*data_size))
        subset_data = Subset(train_data, indices)
        loader = torch.utils.data.DataLoader(subset_data, batch_size=batch_size)
        teacher_loaders.append(loader)
        
    return teacher_loaders

teacher_loaders = get_data_loaders(train_data, num_teachers)

In [11]:
# Create the public dataset by using 90% of the Test data as train #data and remaining 10% as test data.
student_train_data = Subset(test_data, list(range(9000)))
student_test_data = Subset(test_data, list(range(9000, 10000)))

student_train_loader = torch.utils.data.DataLoader(student_train_data, batch_size=batch_size)
student_test_loader = torch.utils.data.DataLoader(student_test_data, batch_size=batch_size)

In [12]:
#Define a simple CNN to classify the MNIST digits.
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

class Classifier(nn.Module):
    """ A Simple Feed Forward Neural Network. 
        A CNN can also be used for this problem 
    """
    def __init__(self):
        super().__init__()
        
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.conv2_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(320, 50)
        self.fc2 = nn.Linear(50, 10)
    
    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        x = x.view(-1, 320)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = self.fc2(x)
        return F.log_softmax(x)

In [13]:
#Now define the train and predict functions
def train(model, trainloader, criterion, optimizer, epochs=10):
    """ This function trains a single Classifier model """
    running_loss = 0
    for e in range(epochs):
        model.train()
        
        for images, labels in trainloader:
            optimizer.zero_grad()
            
            output = model.forward(images)
            loss = criterion(output, labels)
            loss.backward()
            optimizer.step()
            
            running_loss += loss.item()
            
            

def predict(model, dataloader):
    """ This function predicts labels for a dataset 
        given the model and dataloader as inputs. 
    """
    outputs = torch.zeros(0, dtype=torch.long)
    model.eval()
    
    for images, labels in dataloader:
        output = model.forward(images)
        ps = torch.argmax(torch.exp(output), dim=1)
        outputs = torch.cat((outputs, ps))
        
    return outputs

In [15]:
def predict(model, dataloader):
    """ This function predicts labels for a dataset 
        given the model and dataloader as inputs. 
    """
    outputs = torch.zeros(0, dtype=torch.long)
    model.eval()
    
    for images, labels in dataloader:
        output = model.forward(images)
        ps = torch.argmax(torch.exp(output), dim=1)
        outputs = torch.cat((outputs, ps))
        
    return outputs

In [16]:
def train_models(num_teachers):
    """ Trains *num_teacher* models (num_teachers being the number of teacher classifiers) """
    models = []
    for i in range(num_teachers):
        model = Classifier()
        criterion = nn.NLLLoss()
        optimizer = optim.Adam(model.parameters(), lr=0.003)
        train(model, teacher_loaders[i], criterion, optimizer)
        models.append(model)
    return models

In [17]:
models = train_models(num_teachers)

/Users/annascott/miniconda3/envs/pysyft/lib/python3.7/site-packages/ipykernel_launcher.py:26: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


Now, we need to choose the epsilon value for which we first define the formal definition of Differential Privacy

This definition does not create differential privacy, instead, it is a measure of how much privacy is afforded by a query M. Specifically, it’s a comparison between running the query M on a database (x) and a parallel database (y). As you remember, parallel databases are defined to be the same as a full database (x) with one entry/person removed.

Thus, this definition says that FOR ALL parallel databases, the maximum distance between a query on database (x) and the same query on database (y) will be e^epsilon, but that occasionally this constraint won’t hold with probability delta. Thus, this theorem is called “epsilon-delta” differential privacy.

How much noise should we add?

The amount of noise necessary to add to the output of a query is a function of four things:

the type of noise (Gaussian/Laplacian)
the sensitivity of the query/function
the desired epsilon (ε)
the desired delta (δ)
Thus, for each type of noise we’re adding, we have a different way of calculating how much to add as a function of sensitivity, epsilon, and delta. We’re going to focus on Laplacian noise.

Laplacian noise is increased/decreased according to a “scale” parameter b. We choose “b” based on the following formula.

b = sensitivity(query) / epsilon

In other words, if we set b to be this value, then we know that we will have a privacy leakage of <= epsilon. Furthermore, the nice thing about Laplace is that it guarantees this with delta == 0. There are some tunings where we can have very low epsilon where delta is non-zero, but we’ll ignore them for now.



In [18]:
import numpy as np

epsilon = 0.2

In [19]:
def aggregated_teacher(models, dataloader, epsilon):
    """ Take predictions from individual teacher model and 
        creates the true labels for the student after adding 
        laplacian noise to them 
    """
    preds = torch.torch.zeros((len(models), 9000), dtype=torch.long)
    for i, model in enumerate(models):
        results = predict(model, dataloader)
        preds[i] = results
    
    labels = np.array([]).astype(int)
    for image_preds in np.transpose(preds):
        label_counts = np.bincount(image_preds, minlength=10)
        beta = 1 / epsilon

        for i in range(len(label_counts)):
            label_counts[i] += np.random.laplace(0, beta, 1)

        new_label = np.argmax(label_counts)
        labels = np.append(labels, new_label)
    
    return preds.numpy(), labels

In [20]:
teacher_models = models
preds, student_labels = aggregated_teacher(teacher_models, student_train_loader, epsilon)

/Users/annascott/miniconda3/envs/pysyft/lib/python3.7/site-packages/ipykernel_launcher.py:26: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


In [21]:
#Create the Student model and train it using the labels generated above
def student_loader(student_train_loader, labels):
    for i, (data, _) in enumerate(iter(student_train_loader)):
        yield data, torch.from_numpy(labels[i*len(data): (i+1)*len(data)])

In [22]:
student_model = Classifier()
criterion = nn.NLLLoss()
optimizer = optim.Adam(student_model.parameters(), lr=0.003)
epochs = 10
steps = 0
running_loss = 0
for e in range(epochs):
    student_model.train()
    train_loader = student_loader(student_train_loader, student_labels)
    for images, labels in train_loader:
        steps += 1
        
        optimizer.zero_grad()
        output = student_model.forward(images)
        loss = criterion(output, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        
        if steps % 50 == 0:
            test_loss = 0
            accuracy = 0
            student_model.eval()
            with torch.no_grad():
                for images, labels in student_test_loader:
                    log_ps = student_model(images)
                    test_loss += criterion(log_ps, labels).item()
                    
                    # Accuracy
                    ps = torch.exp(log_ps)
                    top_p, top_class = ps.topk(1, dim=1)
                    equals = top_class == labels.view(*top_class.shape)
                    accuracy += torch.mean(equals.type(torch.FloatTensor))
            student_model.train()
            print("Epoch: {}/{}.. ".format(e+1, epochs),
                  "Training Loss: {:.3f}.. ".format(running_loss/len(student_train_loader)),
                  "Test Loss: {:.3f}.. ".format(test_loss/len(student_test_loader)),
                  "Test Accuracy: {:.3f}".format(accuracy/len(student_test_loader)))
            running_loss = 0


/Users/annascott/miniconda3/envs/pysyft/lib/python3.7/site-packages/ipykernel_launcher.py:26: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


Epoch: 1/10..  Training Loss: 0.333..  Test Loss: 1.201..  Test Accuracy: 0.646
Epoch: 1/10..  Training Loss: 0.191..  Test Loss: 0.620..  Test Accuracy: 0.797
Epoch: 1/10..  Training Loss: 0.129..  Test Loss: 0.385..  Test Accuracy: 0.888
Epoch: 1/10..  Training Loss: 0.083..  Test Loss: 0.354..  Test Accuracy: 0.905
Epoch: 1/10..  Training Loss: 0.085..  Test Loss: 0.272..  Test Accuracy: 0.925
Epoch: 2/10..  Training Loss: 0.139..  Test Loss: 0.462..  Test Accuracy: 0.896
Epoch: 2/10..  Training Loss: 0.099..  Test Loss: 0.303..  Test Accuracy: 0.905
Epoch: 2/10..  Training Loss: 0.080..  Test Loss: 0.230..  Test Accuracy: 0.934
Epoch: 2/10..  Training Loss: 0.077..  Test Loss: 0.252..  Test Accuracy: 0.923
Epoch: 2/10..  Training Loss: 0.054..  Test Loss: 0.218..  Test Accuracy: 0.938
Epoch: 2/10..  Training Loss: 0.053..  Test Loss: 0.253..  Test Accuracy: 0.935
Epoch: 3/10..  Training Loss: 0.112..  Test Loss: 0.266..  Test Accuracy: 0.933
Epoch: 3/10..  Training Loss: 0.077..  T

In [23]:
#Perform PATE Analysis on the student labels generated by the Aggregated Teacher
from syft.frameworks.torch.differential_privacy import pate

data_dep_eps, data_ind_eps = pate.perform_analysis(teacher_preds=preds, indices=student_labels, noise_eps=epsilon, delta=1e-5)
print("Data Independent Epsilon:", data_ind_eps)
print("Data Dependent Epsilon:", data_dep_eps)

W0731 22:04:29.687870 4347536832 secure_random.py:26] Falling back to insecure randomness since the required custom op could not be found for the installed version of TensorFlow. Fix this by compiling custom ops. Missing file was '/Users/annascott/miniconda3/envs/pysyft/lib/python3.7/site-packages/tf_encrypted/operations/secure_random/secure_random_module_tf_1.14.0.so'
W0731 22:04:29.703739 4347536832 deprecation_wrapper.py:119] From /Users/annascott/miniconda3/envs/pysyft/lib/python3.7/site-packages/tf_encrypted/session.py:26: The name tf.Session is deprecated. Please use tf.compat.v1.Session instead.



Data Independent Epsilon: 1451.5129254649705
Data Dependent Epsilon: 14.581123135054918


The pate.perform_analysis method returns two values – a data-independent epsilon and a data-dependent epsilon. The data-dependent epsilon is the epsilon value obtained by looking at how much the teachers agree with each other. In a way, the PATE analysis rewards the user for building teacher models which agree with each other because it becomes harder to leak information and track individual information.